# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models, utils

import time
import os
import copy
from collections import OrderedDict

import random
from random import randint
from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
# I do not like to hard code stuff, so that I can re-use it, so I will define constants here:
batch_size = 64

# The following will be used for the comand line application also
archs = ['vgg','alexnet','densenet'] # choice of model architecture
c_arch = archs[0]
num_epochs = 8
learning_rate = 0.01
hidden_units = 1024
use_gpu = True # whether we want to use the gpu or not
c_drop = 0.5 # droupout choice

# in order to match the normalization used when the models were trained, we we need to use the ImageNet standars
# these will be used for our transforms for training, validating and testing data
# these will also be used in the section were we process images
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Image information: In this case, we are using 224x224 images, since it is required by the model.
# I leave it here just in case I ever want to change that
img_px = 224

In [2]:
# directories for the data (Not really used...)
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets

# I defined my transforms with the help of a PyTorch official tutorial
# Link: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

# Note that for the training, we randomize. Also note that we are using our constants from above
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(img_px),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'valid': transforms.Compose([
        transforms.Resize(img_px + 1),
        transforms.CenterCrop(img_px),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(img_px + 1),
        transforms.CenterCrop(img_px),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

# I defined my transforms with the help of a PyTorch official tutorial
# Link: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = batch_size,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'valid', 'test']}

## Sanity Check #1:

I want to do this project step by step, so first I want to make sure the data was loaded correctly and I also want to display useful information.

In [ ]:
# Get the data set sizes:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
print("Dataset Sizes: ", dataset_sizes)
print()

# Get the class names from the training dataset:
class_names = image_datasets['train'].classes
print("Class Names", class_names);
print()

# Get the # of classes (Useful to figure out what the output size for our last layer in the network should be)
# By inspecting the folder, we can see that it should be 102, but I do not want to hard code this
num_classes = len(class_names)
print("Number of classes: ", num_classes);

## Sanity Check #2:

I want to make sure the images were loaded correctly, so I will display some below (will display an actual batch of 64 images):

In [ ]:
# Again, I used code from the PyTorch.org tutorial on transfer learning
# Link: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = utils.make_grid(inputs)
imshow(out, title='Sample Batch')

As we can see, we are displaying flower images, so we are doing good so far. We can move on.

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    
# just for fun (and to verify that all is well, I want to print the contents of cat_to_name)
print("Cat to name: ", cat_to_name);
print()
print('Example mapping for cat 70:', cat_to_name['70'])

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

The model is built from two parts, the features and the classifier. The classifier layer was trained on the ImageNet data set, so it will not work for this project. So we will replace the classifier on our model with our own.

The function below is modified from the PyTorch.org tutorial at:
https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#set-model-parameters-requires-grad-attribute

It is just a simpler way to set the .requires_grad attriute of the parameters of a model to false, without having to do using a for loop
for each model in the initialize_model() function that is defined below.


In [ ]:
def set_parameter_requires_grad(model):
    """
    This simple function just sets the .requires_grad
    attribute for each model parameter to false.
    We do this since we are feature extracting.
    """
    for param in model.parameters():
        param.requires_grad = False

For this portion of the project, I used what we learned in class along with the following tutorial:

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#initialize-and-reshape-the-networks

The main difference between what is shown in the tutorial and what I implement is that in the tutorial they only replace certain layers of the classifier for some of the models, whereas I replace the entire classifier
for the pre-trained model. 

In [ ]:
def initialize_model(model_name, num_classes):
    """Initialize our model."""
    
    model_ft = None
    
    # use if and elif statements to check which one of the pre-trained
    # models we want to use
    if model_name == "alexnet":

        model_ft = models.alexnet(pretrained=True)
        set_parameter_requires_grad(model_ft)
        
        # get the number of input features from the 6th layer
        # of the 'original' classifier
        num_ftrs = model_ft.classifier[6].in_features
        
        # replace the entire classifier
        model_ft.classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(num_ftrs, hidden_units)),
                          ('drop', nn.Dropout(c_drop)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(hidden_units, num_classes)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
        
    elif model_name == "vgg":

        model_ft = models.vgg19(pretrained=True)
        set_parameter_requires_grad(model_ft)
        
        # get the number of input features from the 6th layer
        # of the 'original' classifier
        num_ftrs = model_ft.classifier[6].in_features
        
        # replace the entire classifier
        model_ft.classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(num_ftrs, hidden_units)),
                          ('drop', nn.Dropout(c_drop)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(hidden_units, num_classes)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
        
    elif model_name == "densenet":

        model_ft = models.densenet121(pretrained=True)
        set_parameter_requires_grad(model_ft)
        
        # get the number of input features from
        # of the 'original' classifier
        num_ftrs = model_ft.classifier.in_features
        
        # replace the entire classifier
        model_ft.classifier =  nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(num_ftrs, hidden_units)),
                          ('drop', nn.Dropout(c_drop)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(hidden_units, num_classes)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
            
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft

In [ ]:
# Initialize the model
model_ft = initialize_model(c_arch, num_classes)

# Print the model
print(model_ft)

For training our network, I used a combination of what we learned in class, as well as code from the following PyTorch.org tutorials:

Class repo: https://github.com/udacity/DL_PyTorch

https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html#model-training-and-validation-code

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#training-the-model


In [ ]:
def train_model(model, device, dataloaders, criterion, optimizer, num_epochs=25):
    """Train the network."""    
    since = time.time()

    model.to(device)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # For each epoch, we will go through a training and 
        # validating phase. We will display the loss and
        # accuracy for each.
        for phase in ['train', 'valid']:
            
            # check what phase we are in and set
            # the model to training or evaluate mode
            if phase == 'train':
                model.train()
            else:
                model.eval()

            # reset running loss and running corrects
            running_loss = 0.0
            running_corrects = 0

            # Iterate over train or valid data (depends on
            # the phase that we are in)
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # clear the gradients, since they are
                # accumulated
                optimizer.zero_grad()

                # forward pass through the network
                # turn gradients on only for the training phase
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backpropagation and optimize 
                    # only if in training phase
                    if phase == 'train':
                        loss.backward()
                        # update the weights
                        optimizer.step()

                # get statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            # calculate the loss and accuracy for current epoch phase
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            # in other words, we store the best model
            # only if we get a better validation accuracy
            # than in the previous epoch
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    # print training complete stats
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Detect if we have a GPU available and use it if we chose to in the
# hyperparameters up top. 
device = torch.device("cuda:0" if torch.cuda.is_available() and use_gpu else "cpu")
device

In [ ]:
# Setup the criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.classifier.parameters(), lr=learning_rate)

# Train and evaluate
model_ft = train_model(model_ft, device, dataloaders, criterion, optimizer, num_epochs)

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

The code for checking the accuracy on the test set was found in the
Part8-TransferLearningSolution Jupyter Notebook included with the class.

In [ ]:
def check_accuracy_on_test(model, testloader, device):    
    """Checks the accuracy of our network on the test set."""
    correct = 0
    total = 0
    model.to(device)
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the {} test images:{}%'.format(dataset_sizes['test'], 100 * correct / total))

In [ ]:
check_accuracy_on_test(model_ft, dataloaders['test'], device)

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
def save_model(model, class_to_idx_mapping, path):
    """Saves the checkpoint"""
    model.class_to_idx = class_to_idx_mapping
    m_state_dict = model.state_dict()
    
    checkpoint = {'output_size': num_classes,
                  'choice_model': c_arch,
                  'hidden_unit' : hidden_units,
                  'learning_rate': learning_rate,
                  'model_state_dict': m_state_dict,
                  'class_to_idx': model.class_to_idx}
    
    torch.save(checkpoint, path)
    print('Model saved!')

In [ ]:
save_model(model_ft, image_datasets['train'].class_to_idx, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filename):
    """Loads the checkpoint."""    
    # load parameters
    checkpoint = torch.load(filename)
    c_arch = checkpoint['choice_model']
    num_classes = checkpoint['output_size']
    hidden_unit = checkpoint['hidden_unit']
    learning_rate = checkpoint['learning_rate']
    # load the model
    model = initialize_model(c_arch, num_classes)
    # load model state and class idx
    model.load_state_dict(checkpoint['model_state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']  
    
    # print("model.class_idx", model.class_to_idx)
    
    return model

In [ ]:
# Load the checkpoint:
checkpoint_filename = 'checkpoint.pth'
model_ft = load_checkpoint(checkpoint_filename)

print(model_ft)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image_path):
    """Process an image path into a PyTorch tensor"""

    # open the image
    image = Image.open(image_path)
    
    # Resize the image
    image = image.resize((256, 256))

    # perform center crop
    # https://stackoverflow.com/questions/16646183/crop-an-image-in-the-centre-using-pil
    width, height = image.size   # Get dimensions
    # print("image size 1: ", image.size)
    new_height, new_width = 224, 224
    
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    image = image.crop((left, top, right, bottom))
    # print("image size 2: ", image.size)
    
    # convert encoded color channels and convert to floats (divide by 255)
    np_image = np.array(image) / 255
    # print(np_image)
    
    # normalize
    np_image = (np_image - mean) / std
    
    # finally, transpose
    # print("shape 1: ", np_image.shape)
    np_image = np_image.transpose((2, 0, 1))
    # print("transposed shape: ", np_image.shape)
    
    # Originally, I was returning a numpy array, as I thought these were the instructions, but
    # when trying to test, it would not work. 
    # Found solution at: https://knowledge.udacity.com/questions/29173
    # We have to convert to a tensor before we return it
    return torch.Tensor(np_image)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
def get_random_image_path():
    """ 
    Get a random image path from the flowers
    Test Folder. I will use this function to
    use the predict function on any random image
    in that folder. 
    """
    
    # get a random index for the folder (there are 102)
    rand_folder_idx = randint(1, 102)
    # print(rand_folder_idx)

    # get a random image
    # build the dir/path
    rand_dir = 'flowers/test/' + str(rand_folder_idx) + '/' 

    # get a random file from the dir
    # https://stackoverflow.com/questions/701402/best-way-to-choose-a-random-file-from-a-directory
    rand_img = random.choice(os.listdir(rand_dir))

    # build fuild image path
    rand_img_path = rand_dir + rand_img
    
    return rand_img_path

In [ ]:
# test random image function
rand_img_path = get_random_image_path()
img = process_image(rand_img_path)
imshow(img)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, device, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    # set the mode for inference
    model.eval()
    
    # set the device
    model.to(device)
    
    # process the image
    image = process_image(image_path);
    image = np.expand_dims(image, 0)
    
    img_to_fwd = torch.from_numpy(image)
    img_to_fwd = img_to_fwd.to(device)
    
    # Turn off gradients to speed up this part
    with torch.no_grad():
        # fwd pass get logits
        output = model.forward(img_to_fwd)

    # Calculate the class probabilities for img
    # ps = torch.exp(output)
    # Calculate the class probabilities (softmax) for img
    ps = F.softmax(output, dim=1)
    # get the top K largest values
    probs, classes = ps.topk(5)
    
    # probs and classes are tensors, so we convert to lists so we return
    # as is required
    top_probs = probs.cpu().detach().numpy().tolist()[0]
    top_classes = classes.cpu().detach().numpy().tolist()[0]
    
    # I was getting the wrong class labels when converting,
    # the solution in the following helped me:
    # https://knowledge.udacity.com/questions/31597
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    
    classes = []
    
    # convert the classes using idx_to_class
    for cls in top_classes:
        c = idx_to_class[cls]
        classes.append(c)
    
    # return the 
    return top_probs, classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

For this part, especially for plotting more than one plot, I used the code from the view_classify() function in the helper.py file from the class repo: https://github.com/udacity/DL_PyTorch

In [ ]:
# get a random image from a random path
random_image = get_random_image_path()
print("predicting image: ", random_image)

# predict, and get probabilities and classes
probabilities, classes = predict(random_image, model_ft, device)
print("Probs: ", probabilities)
print()
print("Classes: ", classes)

# To plot more than one sublot
# I used https://github.com/udacity/DL_PyTorch/blob/master/helper.py
# As a starting point.
fig, (ax1, ax2) = plt.subplots(figsize=(6,8), nrows=2)

fig.subplots_adjust(hspace=0.1, wspace=0.1)

# get the index of the maximum probability
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html
max_index = np.argmax(probabilities)

# get the label for the class with the max probability
max_index_label = cat_to_name[classes[max_index]]

# We 'plot' the image
# open the image
image = Image.open(random_image)

# turn axis off, set the title and show the image
ax1.axis('off')
ax1.set_title(max_index_label)
ax1.imshow(image)

# create all the labels by converting using the cat_to_name
# mapping
labels = []
for cls in classes:
    labels.append(cat_to_name[cls])

# show the bar graph    
y_pos = np.arange(len(labels))
ax2.barh(y_pos, probabilities)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(labels)
ax2.set_xlim(0, 1.0)

plt.tight_layout()